In [4]:
#Varibales for Code

df=spark.createDataFrame([
["AMD", "COK", "2024-02-24 08:00:00", "2024-02-24 10:30:00", "2024-02-24 08:50:00", "2024-02-24 11:25:00", "Akasa Air", "airTraffic"],
["COK", "SXR", "2024-02-25 12:00:00", "2024-02-25 18:30:00", "2024-02-25 12:15:00", "2024-02-25 19:15:00", "Vistara", "mechanical"],
["COK", "AMD", "2024-03-24 08:00:00", "2024-03-24 10:40:00", "2024-03-24 08:17:00", "2024-03-24 11:00:00", "Akasa Air", "storm"],
["SXR", "AMD", "2024-04-28 05:30:00", "2024-04-28 07:35:00", "2024-04-28 05:50:00", "2024-04-28 07:55:00", "Vistara", "visibility"],
["AMD", "SXR", "2024-07-25 07:00:00", "2024-07-25 09:10:00", "2024-07-25 07:10:00", "2024-07-25 09:20:00", "Akasa Air", "airTraffic"],
["AMD", "CCU", "2024-03-28 18:00:00", "2024-03-28 20:25:00", "2024-03-28 19:00:00", "2024-03-28 21:15:00", "Vistara", "storm"],
["SXR", "COK", "2024-06-28 23:00:00", "2024-06-29 04:40:00", "2024-06-28 23:40:00", "2024-06-29 05:20:00", "Vistara", "airTraffic"],
["COK", "CCU", "2024-06-24 18:00:00", "2024-06-24 21:05:00", "2024-06-24 18:05:00", "2024-06-24 21:25:00", "IndiGo", "visibility"],
["CCU", "AMD", "2024-07-25 04:00:00", "2024-07-25 06:45:00", "2024-07-25 04:30:00", "2024-07-25 07:05:00", "IndiGo", "airTraffic"],
["CCU", "COK", "2024-02-28 14:00:00", "2024-02-28 17:10:00", "2024-02-28 14:10:00", "2024-02-28 17:40:00", "IndiGo", "mechanical"],
["CCU", "SXR", "2024-04-21 17:00:00", "2024-04-21 19:50:00", "2024-04-21 17:30:00", "2024-04-21 20:05:00", "IndiGo", "airTraffic"]],
schema=["origin_airport", "destination_airport", "scheduled_departure_time", "scheduled_arrival_time", "actual_departure_time", "actual_arrival_time", "carrier", "delay_reason"])

In [ ]:
#Schema to be imposed
"origin_airport", "destination_airport", "scheduled_departure_time", "scheduled_arrival_time", "actual_departure_time", "actual_arrival_time", "carrier", "delay_reason"

In [16]:
#Import Libraries
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,StructField,TimestampType,StringType

In [3]:
#Create a SparkSession
spark = SparkSession.builder.appName('flight_details').getOrCreate()

In [6]:
#Create dataframe for the given data using enforced Schema method
df.show()

+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+---------+------------+
|origin_airport|destination_airport|scheduled_departure_time|scheduled_arrival_time|actual_departure_time|actual_arrival_time|  carrier|delay_reason|
+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+---------+------------+
|           AMD|                COK|     2024-02-24 08:00:00|   2024-02-24 10:30:00|  2024-02-24 08:50:00|2024-02-24 11:25:00|Akasa Air|  airTraffic|
|           COK|                SXR|     2024-02-25 12:00:00|   2024-02-25 18:30:00|  2024-02-25 12:15:00|2024-02-25 19:15:00|  Vistara|  mechanical|
|           COK|                AMD|     2024-03-24 08:00:00|   2024-03-24 10:40:00|  2024-03-24 08:17:00|2024-03-24 11:00:00|Akasa Air|       storm|
|           SXR|                AMD|     2024-04-28 05:30:00|   2024-04-28 07:35:00|  2024-04-28 05:

In [18]:
#Filter the dataframe to display all delayed flights
arr=df.withColumn('delay_departure',(unix_timestamp('actual_departure_time')-unix_timestamp('scheduled_departure_time'))/60)
arr.filter(col('delay_departure')>0).show()

+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+---------+------------+---------------+
|origin_airport|destination_airport|scheduled_departure_time|scheduled_arrival_time|actual_departure_time|actual_arrival_time|  carrier|delay_reason|delay_departure|
+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+---------+------------+---------------+
|           AMD|                COK|     2024-02-24 08:00:00|   2024-02-24 10:30:00|  2024-02-24 08:50:00|2024-02-24 11:25:00|Akasa Air|  airTraffic|           50.0|
|           COK|                SXR|     2024-02-25 12:00:00|   2024-02-25 18:30:00|  2024-02-25 12:15:00|2024-02-25 19:15:00|  Vistara|  mechanical|           15.0|
|           COK|                AMD|     2024-03-24 08:00:00|   2024-03-24 10:40:00|  2024-03-24 08:17:00|2024-03-24 11:00:00|Akasa Air|       storm|           17.0|
|   

In [20]:
#Calculate Average delay per source airport
arr1=df.groupBy('origin_airport').agg({'delay_departure':'avg'}).show()

+--------------+--------------------+
|origin_airport|avg(delay_departure)|
+--------------+--------------------+
|           SXR|                30.0|
|           AMD|                40.0|
|           COK|  12.333333333333334|
|           CCU|  23.333333333333332|
+--------------+--------------------+



In [22]:
##Calculate Average delay per destination airport
arr2=df.groupBy('destination_airport').agg({'delay_departure':'avg'}).show()

+-------------------+--------------------+
|destination_airport|avg(delay_departure)|
+-------------------+--------------------+
|                SXR|  18.333333333333332|
|                AMD|  22.333333333333332|
|                COK|  33.333333333333336|
|                CCU|                32.5|
+-------------------+--------------------+



In [24]:
#Calculate Average delay per airline
arr3=df.groupBy('carrier').agg({'delay_departure':'avg'}).show()

+---------+--------------------+
|  carrier|avg(delay_departure)|
+---------+--------------------+
|Akasa Air|  25.666666666666668|
|  Vistara|               33.75|
|   IndiGo|               18.75|
+---------+--------------------+



In [28]:
#Fetch airline with maximum delay (in single flight)
arr4=df.select(['carrier','delay_departure']).orderBy('delay_departure',ascending=False).show(1)

+-------+---------------+
|carrier|delay_departure|
+-------+---------------+
|Vistara|           60.0|
+-------+---------------+
only showing top 1 row



In [31]:
#Fetch airline with maximum delay (overall time delay)
arr5=df.groupBy('carrier').agg({'delay_departure':'sum'}).orderBy('sum(delay_departure)',ascending=False).show(1)

+-------+--------------------+
|carrier|sum(delay_departure)|
+-------+--------------------+
|Vistara|               135.0|
+-------+--------------------+
only showing top 1 row



In [32]:
#Fetch airline with minimum delay (in single flight)
arr6= df.select(['carrier','delay_departure']).orderBy('delay_departure',ascending=True).show(1)

+-------+---------------+
|carrier|delay_departure|
+-------+---------------+
| IndiGo|            5.0|
+-------+---------------+
only showing top 1 row



In [35]:
#Fetch airline with minimum delay (overall time delay)
arr7=df.groupBy('carrier').agg(sum('delay_departure')).orderBy('sum(delay_departure)',ascending=True).show(1)

+-------+--------------------+
|carrier|sum(delay_departure)|
+-------+--------------------+
| IndiGo|                75.0|
+-------+--------------------+
only showing top 1 row



In [39]:
#Calculate Average delay per reason
arr8=df.groupBy('delay_reason').agg(avg('delay_departure')).show()

+------------+--------------------+
|delay_reason|avg(delay_departure)|
+------------+--------------------+
|  visibility|                12.5|
|  airTraffic|                32.0|
|       storm|                38.5|
|  mechanical|                12.5|
+------------+--------------------+



In [41]:
#Find most recurring cause of delays
arr9=df.groupBy('delay_reason').agg(count('delay_reason')).orderBy('count(delay_reason)',ascending=False).show(1)

+------------+-------------------+
|delay_reason|count(delay_reason)|
+------------+-------------------+
|  airTraffic|                  5|
+------------+-------------------+
only showing top 1 row



In [42]:
df.show()

+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+---------+------------+---------------+
|origin_airport|destination_airport|scheduled_departure_time|scheduled_arrival_time|actual_departure_time|actual_arrival_time|  carrier|delay_reason|delay_departure|
+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+---------+------------+---------------+
|           AMD|                COK|     2024-02-24 08:00:00|   2024-02-24 10:30:00|  2024-02-24 08:50:00|2024-02-24 11:25:00|Akasa Air|  airTraffic|           50.0|
|           COK|                SXR|     2024-02-25 12:00:00|   2024-02-25 18:30:00|  2024-02-25 12:15:00|2024-02-25 19:15:00|  Vistara|  mechanical|           15.0|
|           COK|                AMD|     2024-03-24 08:00:00|   2024-03-24 10:40:00|  2024-03-24 08:17:00|2024-03-24 11:00:00|Akasa Air|       storm|           17.0|
|   

In [ ]:
#Identify the airline with best recovery time
# (Even though the flight started late, the airline recovered a few minutes in flight time)


